In [ ]:
import os
import shutil                   # 파일 복사와 같은 고수준 연산 모듈
import tqdm
import json
import yaml
from pathlib import Path

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install import-ipynb
%pip install koreanize-matplotlib
%pip install omegaconf
!pip install ultralytics

In [29]:
train_img_paht = './data/ai02-level1-project/images'
new_path = './data/train/images'

os.makedirs(new_path, exist_ok=True)

for train_img in os.listdir(train_img_paht):
    old_img_path = os.path.join(train_img_paht, train_img)
    new_img_path = os.path.join(new_path, train_img)
    shutil.copy(old_img_path, new_img_path)

# 하나의 폴더에 JSON 파일들을 다 넣어줌

In [30]:
# data.yaml, labels txt 파일을 class_map
annotations_path = './data/ai02-level1-project/train_annotations'

# labels txt 파일들 저장할 경로
save_dir = './data/train/annotations'
class_map = []

In [23]:
# cx, cy, w, h 함수
def coco_to_yolo(bbox, img_w, img_h):
    if len(bbox) == 1 and isinstance(bbox[0], (list, tuple)):
        bbox = bbox[0]
    x, y, w, h = bbox
    # 1) 중심 좌표 계산
    cx = x + w / 2
    cy = y + h / 2
    # 2) 정규화
    nx = cx / img_w
    ny = cy / img_h
    nw = w  / img_w
    nh = h  / img_h
    return (nx, ny, nw, nh)

# labels txt 파일

In [31]:
# labels txt 파일 생성
# annotations_path json 순회
os.makedirs(save_dir, exist_ok=True)

for root, dirs, files in os.walk(annotations_path):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
              data = json.load(f)
            file_name = data['images'][0]['imgfile']
            file_name = os.path.splitext(file_name)[0] # (basename, ext) 튜플 반환함
            class_name = data['images'][0]['dl_name']

            # 클래스 이름 클래스 맵에 넣어주기.
            if class_name not in class_map:
              class_map.append(class_name)
            class_idx = class_map.index(class_name)

            # YOLO 포맷 수치 계산
            bbox = data['annotations'][0]['bbox']
            img_w = data['images'][0]['width']
            img_h = data['images'][0]['height']
            cx, cy, w, h = coco_to_yolo(bbox, img_w, img_h)

            # txt 파일 경로
            txt_path = os.path.join(save_dir, f'{file_name}.txt')

            # 쓰기
            with open(txt_path, 'a', encoding='utf-8') as f:
              f.write(f'{class_idx} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n')

In [33]:
annotations_count = len(os.listdir('.\\data\\train/annotations'))
img_count = len(os.listdir('.\\data\\train/images'))

print(f"어노테이션 파일 수:{annotations_count}")
print(f"이미지 수: {img_count}")

어노테이션 파일 수:1489
이미지 수: 1489


# data.yaml

In [18]:
# data.yaml 파일 생성
# data.yaml 파일 생성할 경로
yaml_path = './data/data.yaml'
Path(yaml_path).parent.mkdir(parents=True, exist_ok=True)

class_arr = [None] * len(class_map)
for idx, key in enumerate(class_map):
  class_arr[idx] = key

#
data = {
    'train': './data/train/images',
    'nc': len(class_map),
    'names': class_arr
}

with open(yaml_path, 'w', encoding='utf-8') as f:
  yaml.dump(data, f, allow_unicode=True)